In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import pickle
import skimage

In [ ]:
Categories = ['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'श', 'ष', 'स', 'ह', 'क्ष', 'त्र', 'ज्ञ','del','nothing','space']

In [ ]:
datadir = '../nsl_alphabet'

In [ ]:
flat_data_arr=[] #input array
target_arr=[] #output array

In [ ]:
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
#         edges = cv.Canny(img_array,100,200) #canney edhe detecton
#         cv.imwrite("./data/"+str(c)+".jpg",edges)
        img_resized=resize(img_array,(64,64,3))        
        flat_data_arr.append(img_resized.flatten())        
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')


In [ ]:
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)


In [ ]:
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x=df.iloc[:,:-1] #input data 
y=df.iloc[:,-1] #output data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)

In [ ]:
X_train = np.array(x_train)
X_test = np.array(x_test)
Y_train = np.array(y_train)
Y_test = np.array(y_test)

In [ ]:
XX_train = X_train.reshape((609,64,64,3))
XX_test = X_test.reshape((261,64,64,3))

In [ ]:
plt.imshow(XX_test[0])
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical
y_cat_train = to_categorical(list(Y_train),39)
y_cat_test = to_categorical(Y_test,39)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
print("Packages imported...")

In [ ]:
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(39, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
XX_train.shape

In [ ]:
model.fit(XX_train, y_cat_train,
          epochs=20,
          batch_size=64,
          verbose=2,
          validation_data=(XX_test, y_cat_test),
         callbacks=[early_stop])

In [ ]:
y_pred=model.predict(XX_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(Y_test))
# print(f"The model is {accuracy_score(y_pred,Y_test)*100}% accurate")

In [ ]:
# predictions = model.predict_classes(XX_test)
predictions = (model.predict(XX_test) > 0.5).astype("int32")
print("Predictions done...")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_cat_test,predictions))

In [ ]:
pred = np.argmax(predictions, axis=1)

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,12))
sns.heatmap(confusion_matrix(Y_test,pred))
plt.show()

In [ ]:
import pickle

In [ ]:
model.save('NSL.h5')


In [ ]:
# from keras.models import load_model
# pred_model = load_model("./NSL.h5")

In [ ]:
# a = XX_test[0].reshape((1,64,64,3))
# # pred_model.predict(XX_test)

In [ ]:
# pred_model.predict(a)

In [ ]:
# np.argmax(pred_model.predict(a))